# How to Experimentos:

- Supondo ambiente CloudLab.us

- Criar a topologia (pode ser utilizando o profile.py - se estiver implementado)

- cada host executa automaticamente o script (caso não tenha executado, fazer manualmente)
` wget -o- https://github.com/NiltonMocelin/FLOWPRI-SDN2/archive/refs/heads/main.zip && unzip main.zip && mv FLOWPRI-SDN2-main FLOWPRI-SDN2 && cd FLOWPRI-SDN2 && sh requirements.sh`

- Assim, todos os requisitos são instalados nos hosts.

- prox passo, ajustar a configuração inicial do controlador e dos switches para cada domínio.

<hr/>

- No cloudlab os ips de cada host são pré-configurados na topologia.

- Nos hosts switch ovs -> adaptar o script ovs_host/script.sh , para as interfaces existentes no host para a topologia definida. E Então, executar o script (demora um pouco, pois ocorre compilação)

- Com isso, os "switches" (que são hosts convencionais executando o processo OVS) estão prontos.

- Dessa forma, sabemos as portas e seus endereços, que dizem qual porta alcança o quê.

- Configurar o arquivo cfg para cada domínio, configurando os switches e rotas (fazemos de modo estático por enquanto) e copiar para FLOWPRI-SDN2/ (pois é de onde o controlador carrega o arquivo)

- os arquivos cfg: observar os ips das interfaces dos switches e quem eles alcançam, para escrever as rotas. Também se atentar ao ip do controlador, o id do switch configurado...

- Assim, os nós devem ser capazes de alcançar um ao outro utilizando o controlador (sh run_flowpri2.sh)

- Cada vez que for realizar um experimento, precisa preencher o cfg do dominio corretamente, pois o nome das interfaces pode alterar (com o profile python acho que poderia fixar, mas não dará tempo para isso)

<hr/>

#### Setup host switch:

- rodar o script em CloudLab/ovs_host

- passar os ips para a interface "switch" criada: ip addr del <ip/mask> dev <interface> e ip addr add <ip/mask> dev <interface>

- modificar as rotas para que os pacotes sejam encaminhados para a interface switch

- (isso tem que rodar pelo script ) ...


#### Problema 1 CloudLab: (resolvido)

- Problema é o seguinte, as rotas estão erradas nos switches - precisa arrumar isso, uma opção é usando "ip netns", a outra opção é organizar manualemente

- Existem várias entrada em "route -n" que ligam um IP para uma interface de saída. Então os pacotes são enviados para a primeira ocorrencia da tabela. Uma solução é criar a rota corretamente usando "sudo ip route add ip-destino dev interface-saida"

- Para isso, precisa-se saber qual interface foi conectada a cada host;;; Por azar, o Cloudlab faz meio randômico. No entanto, na topologia foi descrito o endereço IP que conecta cada interface do switch com o host, então usando alguns truques de bash é possível descobrir qual interface liga quem.

- ISso foi corrigido no ovs_host/requirements_ovs_domX.sh do respectivo domínio ! para facilitar.

-- To achando que não precisa disso, pois depois de configurar o OVS, não vai importar mais.

#### Problema 2 CloudLab: (NAO resolvido)

- CloudLab não está conseguindo criar links entre duas regiões diferentes...


#### Problema 3 CloudLab: (NAO resolvido)

- Os scripts iniciais dos hosts não estão sendo executados automáticamente mais.

- Antes executavam, mas agora, pararam.

- Deve ser simples, vou tentar apagar o comando e criar novamente na topologia... Vai entender o que acontece...






Comando utilizados no ovs-switch que funcionou

1  ls
    2  ip a
    3  wget -o- https://github.com/NiltonMocelin/FLOWPRI-SDN2/archive/refs/heads/main.zip && unzip main.zip && mv FLOWPRI-SDN2-main FLOWPRI-SDN2 && cd FLOWPRI-SDN2 && sh requirements_controller_host_switch.sh
    4  wget -o- https://github.com/NiltonMocelin/FLOWPRI-SDN2/archive/refs/heads/main.zip && unzip main.zip && mv FLOWPRI-SDN2-main FLOWPRI-SDN2 && cd FLOWPRI-SDN2 && sh requirements.sh
    5  ip a
    6  sudo ovs-vsctl add-br switch          -- set bridge switch other-config:datapath-id=0000000000000001          -- add-port switch eth1 -- set interface eth1 ofport_request=1          -- add-port switch eth2 -- set interface eth2 ofport_request=2          -- add-port switch eth3 -- set interface eth3 ofport_request=3          -- set-controller switch tcp:172.16.1.10:6653
    7  ip a
    8  ip a > ipa.txt
    9  sud otcpdump -i eth1
   10  sudo tcpdump -i eth1
   11  route -n
   12  arp -n
   13  ping 172.16.1.30
   14  ip a
   15  sudo arp -s 172.16.1.10 02:43:b1:6a:a0:15
   16  sudo arp -s 172.16.1.30 02:61:2d:db:ae:69
   17  route -n
   18  ip a
   19  sudo tcpdump -i eth1 icmp
   20  ip a
   21  sudo ovs-vsctl show
   22  sudo ovs-ofctl dump-flows switch
   23  sudo ovs-ofctl add-flow switch actions=NORMAL
   24  sudo ovs-ofctl dump-flows switch
   25  sudo ovs-ofctl del-flows switch
   26  sudo ovs-ofctl add-flow switch actions=LOCAL
   27  sudo ovs-ofctl del-flows switch
   28  ip addr del eth1
   29  ip addr del dev eth1
   30  sudo ip addr del dev eth1
   31  sudo ifconfig eth1 0
   32  sudo ifconfig eth2 0
   33  sudo ifconfig eth3 0
   34  sudo ip addr add
   35  sudo ip addr add 172.16.1.40/24 dev switch
   36  sudo ip addr add 172.16.1.20/24 dev switch
   37  sudo ip addr add 172.16.1.50/24 dev switch
   38  ip a
   39  sudo ip link
   40  sudo ip link set switch up
   41  sudo ip link
   42  ip a
   43  sudo ovs-ofctl dump-flows switch
   44  sudo ovs-ofctl add-flow switch actions=NORMAL
   45  sudo ovs-ofctl dump-flows switch
   46  route -n
   47  sudo ovs-vsctl del-port switch eth2
   48  sudo tcpdump -i eth1
   49  sudo tcpdump -i eth2
   50  sudo tcpdump -i eth3
   51  sudo tcpdump -i switch
   52  ip r
   53  ip a
   54  ip addr del 172.16.1.20/24 dev switch
   55  sudo ip addr del 172.16.1.20/24 dev switch
   56  sudo ip addr del 172.16.1.50/24 dev switch
   57  ip a
   58  route -n
   59  sudo ip route add 172.16.1.20 vi 172.16.1.40 dev switch
   60  sudo ip route add 172.16.1.20 via 172.16.1.40 dev switch
   61  sudo ip route add 172.16.1.50 via 172.16.1.40 dev switch
   62  sudo ip route add 172.16.1.40 via 172.16.1.40 dev switch
   63  ip route show
   64  sudo ovs-ofctl dump-flows switch
   65  sudo ovs-ofctl add-flows switch actions=LOCAL
   66  sudo ovs-ofctl add-flow switch actions=LOCAL
   67  sudo ovs-ofctl dump-flows switch
   68  sudo ovs-ofctl add-flow switch actions=NORMAL
   69  ip route show
   70  iconfig
   71  ifconfig
   72  ip link
   73  ip a
   74  sudo ovs-vsctl del-br switch
   75  sudo ip addr add 172.16.1.20 dev eth1
   76  ping 172.16.1.30
   77  sudo ovs-vsctl add-br switch
   78  sudo ovs-vsctl add-port switch eth1
   79  sudo ovs-ofctl dump-flows switch
   80  ip a
   81  sudo ip link set switch up
   82  sudo ovs-ofctl dump-flows switch
   83  sudo ifconfig eth1 0
   84  sudo ovs-ofctl dump-flows switch
   85  sudo ifconfig switch 172.16.1.20/24
   86  sudo ovs-ofctl dump-flows switch
   87  sudo ovs-ofctl add-flow switch actions=LOCAL
   88  sudo ovs-ofctl dump-flows switch
   89  sudo ovs-ofctl add-flow switch actions=NORMAL
   90  watch sudo ovs-ofctl dump-flows switch
   91  sudo ovs-vsctl add-port switch eth2
   92  watch sudo ovs-ofctl dump-flows switch
   93  watch sudo ovs-ofctl add-flow switch in_port=1,actions=outport=2
   94  sudo ovs-ofctl add-flow switch in_port=1,actions=outport:2
   95  sudo ovs-ofctl add-flow switch in_port=1,actions:outport=2
   96  watch sudo ovs-ofctl dump-flows switch
   97  sudo ovs-vsctl del-flows switch
   98  sudo ovs-ofctl del-flows switch
   99  sudo ovs-ofctl add-flow switch in_port=1,actions:outport=2
  100  ip a
  101  sudo ovs-ofctl add-flow switch in_port=2,actions:outport=1
  102  watch sudo ovs-ofctl dump-flows switch
  103  # hosts nao se respondem ... mal configuração, provavelmente culpa minha..
  104  watch sudo ovs-ofctl dump-flows switch
  105  history 
